In [1]:
from openai import OpenAI
import json
import os
import csv

In [2]:
client=OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
def dataCleanup(fileLocation):
    questions=[]
    with open(fileLocation,mode="r",encoding='utf-8') as csv_file:
        csv_reader=csv.DictReader(csv_file)
        for row in csv_reader:
            questions.append(row["question1"])
    return questions

questions=dataCleanup("./resources/questions.csv")
length=len(questions)
print(length)

30


In [5]:
answer_prompt="""
provide a short,and precise response for the latest query as if you are a university teacher.
"""

In [6]:
question_prompt="""
The following is a piece of dialogue between a student and a teacher.\n
Ask a question based on the context and content of the dialogue as if you where an undergraduate electrical and electronic engineering student asking your professor.\n
The question should be insightful, short, and diverse.\n

You will be penalized 1999 dollars if you ask question that is same or similar to any previous question.\n
You will be penalized 2999 dollars if you provide feedback instead of a question.\n
Avoid any kind of addressing.\n\n
"""

In [7]:
conversationLevel=4
# questions=["Describe the concept of *thermal runaway* in IGBT devices"]
for idx,question in enumerate(questions):
    # if idx<=300:
    #     continue
    converationArray=[]

    conversationHistoryAnswering=[
        {
        "role": "system",
        "content": answer_prompt, 
        },
        {
        "role": "user",
        "content": question,
        }
    ]
    
    # For generating questions
    conversationHistoryQuestioning=[
        {
        "role": "system",
        "content": question_prompt,
        },
        {
        "role": "user",
        "content": question,
        }
    ]

    
    currentQuestion=question
    for p in range(conversationLevel):

        #TEACHER PART........................................................................
        teacher = client.chat.completions.create(
            messages=conversationHistoryAnswering,
            model="gpt-4o",
        )
        teacherAnswer = teacher.choices[0].message.content


        #append the teacher answer.......................
        conversationHistoryAnswering.append({
            "role": "assistant",
            "content": teacherAnswer,
        })
        conversationHistoryQuestioning.append({
            "role": "assistant",
            "content": teacherAnswer,
        })


        #STUDENT PART........................................................................
        student = client.chat.completions.create(
            messages=conversationHistoryQuestioning,
            model="gpt-4o",
        )
        studentQuestion = student.choices[0].message.content

        conversationHistoryAnswering.append({
            "role": "user",
            "content": studentQuestion,
        })
        conversationHistoryQuestioning.append({
            "role": "user",
            "content": studentQuestion,
        })
        print("LEVEL ",p,"\n======================================================================")
        print("Student : ",currentQuestion)
        print("Teacher : ",teacherAnswer)
        currentQuestion=studentQuestion
        
        converationArray.append(teacherAnswer)
        converationArray.append(studentQuestion)
    data = {
        "question1": question,
        "answer1": converationArray[0],
        "question2": converationArray[1],
        "answer2": converationArray[2],
        "question3": converationArray[3],
        "answer3": converationArray[4],
        "question4": converationArray[5],
        "answer4": converationArray[6],
    }
    filename = "output.csv"

    # Define the fieldnames for the CSV file
    fieldnames = ["question1", "answer1", "question2","answer2","question3","answer3","question4","answer4"]

    # Check if the file exists
    file_exists = os.path.isfile(filename)

    # Write data to CSV file
    with open(filename, 'a', newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
        # Write header if file doesn't exist
        if not file_exists:
            writer.writeheader()
    
        # Write data row
        writer.writerow(data)

LEVEL  0 
Student :  What is a binary number system and how does it differ from the decimal system?
Teacher :  The binary number system is a base-2 numeral system that uses only two digits, 0 and 1, to represent values. It is used extensively in computers and digital systems because it aligns with the on/off states of electronic circuits. 

In contrast, the decimal system is a base-10 numeral system that uses ten digits, from 0 to 9, to represent values. The main difference lies in their bases: binary (base-2) and decimal (base-10). Consequently, operations and number representations differ between the two systems.
LEVEL  1 
Student :  Can you elaborate on how binary arithmetic, particularly multiplication and division, is implemented in digital circuits compared to decimal operations?
Teacher :  Certainly. In digital circuits, binary arithmetic operations like multiplication and division are implemented using logic gates, which handle binary digits (bits).

**Binary Multiplication**: 